In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow.compat.v2 as tf2
import tensorflow_datasets as tfds


In [2]:
(train_data, test_data), info = tfds.load(
    'imdb_reviews/subwords8k', 
    split = (tfds.Split.TRAIN, tfds.Split.TEST), 
    with_info=True, as_supervised=True)

In [9]:
encoder = info.features['text'].encoder
train_batches = train_data.shuffle(1000).padded_batch(10)
test_batches = test_data.shuffle(1000).padded_batch(10)
encoder.subwords[:20]

['the_',
 ', ',
 '. ',
 'a_',
 'and_',
 'of_',
 'to_',
 's_',
 'is_',
 'br',
 'in_',
 'I_',
 'that_',
 'this_',
 'it_',
 ' /><',
 ' />',
 'was_',
 'The_',
 'as_']

In [54]:
max_input_len = encoder.vocab_size
encoded_dims = 16
inputs = layers.Input(shape=(max_input_len))
embed = layers.Embedding( encoder.vocab_size, encoded_dims)(inputs)
print(embed.shape)
avgpool = layers.GlobalAveragePooling1D()(embed) 
print(avgpool.shape)
dense1 = layers.Dense(16, activation='relu')(avgpool)
print( dense.shape)
dense2 = layers.Dense(1)(dense1)
print( dense2.shape)


(None, 8185, 16)
(None, 16)
(None, 16)
(None, 1)


In [55]:
model = keras.Model(inputs=inputs, outputs=dense2)
print(model.summary())

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_23 (InputLayer)        [(None, 8185)]            0         
_________________________________________________________________
embedding_28 (Embedding)     (None, 8185, 16)          130960    
_________________________________________________________________
global_average_pooling1d_15  (None, 16)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 17        
Total params: 131,249
Trainable params: 131,249
Non-trainable params: 0
_________________________________________________________________
None


In [57]:
model.compile( optimizer='adam', loss=keras.losses.BinaryCrossentropy(from_logits=True))
model.fit( train_batches, epochs = 10, validation_data=test_batches, validation_steps = 20)

Epoch 1/10
2500/2500 [==============================] - 5s 2ms/step - loss: 0.6136 - val_loss: 0.3980
Epoch 2/10
2500/2500 [==============================] - 5s 2ms/step - loss: 0.3054 - val_loss: 0.4026
Epoch 3/10
2500/2500 [==============================] - 5s 2ms/step - loss: 0.2412 - val_loss: 0.3750
Epoch 4/10
2500/2500 [==============================] - 5s 2ms/step - loss: 0.2103 - val_loss: 0.2764
Epoch 5/10
2500/2500 [==============================] - 5s 2ms/step - loss: 0.1835 - val_loss: 0.4777
Epoch 6/10
2500/2500 [==============================] - 5s 2ms/step - loss: 0.1667 - val_loss: 0.3593
Epoch 7/10
2500/2500 [==============================] - 5s 2ms/step - loss: 0.1473 - val_loss: 0.4044
Epoch 8/10
2500/2500 [==============================] - 5s 2ms/step - loss: 0.1326 - val_loss: 0.3627
Epoch 9/10
2500/2500 [==============================] - 5s 2ms/step - loss: 0.1239 - val_loss: 0.5353
Epoch 10/10
2500/2500 [==============================] - 5s 2ms/step - loss: 0.111